In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from scipy import stats
from scipy import interpolate
import matplotlib.cm as cm
import numpy.polynomial.polynomial as poly
%matplotlib inline

In [2]:
df = pd.read_csv('../WORK/surf_bT_ret_171031.txt', header=None, delimiter=r"\s+", names = [' ', 'UTC', 'MY', 'L_s', 'LTST', 'Surf_lat', 'Surf_lon', 'Surf_rad', 'Surf_elev', 'T_surf', 'T_surf_err', 'Dust_column', 'Dust_column_err', 'H2Oice_column', 'H2Oice_column_err', 'p_surf', 'p_surf_err', 'P_qual', 'T_qual'])
#read data into dataframe

The above reads the text file into a dataframe

In [3]:
########
# pco2 #
########
# --------------------------------------------
# Equilibrium vapor pressure over solid CO2
# Brown and Ziegler (1980)
# --------------------------------------------
# Input:
#    T = temperature of solid [K]
# Output:
#    vapor pressure [Pa]
def pco2(T):
    A0 = 2.13807649e1
    A1 = -2.57064700e3
    A2 = -7.78129489e4
    A3 = 4.32506256e6
    A4 = -1.20671368e8
    A5 = 1.34966306e9
    # Pressure in torr
    ptorr = np.exp(A0 + (A1/(T)) + (A2/(T**2)) + (A3/(T**3)) + (A4/(T**4)) + (A5/(T**5)))
    # Pressure in Pa
    p = ptorr*133.3223684211
    return p

In [4]:
########
# tco2 #
########
#-------------------------------------------------------------------------
#Calculate (approximately) the condensation temperature of CO2 at the
#specified pressure(s) (Pa), using the empirical law from Brown and 
#Ziegler (1980). This function calls the pco2() routine.
#-------------------------------------------------------------------------
# Input:
#    p = vapor pressure [Pa]
# Output:
#    temperature of solid [K]
def tco2(p):
    Trange = np.arange(30,350)
    T = interpolate.pchip_interpolate(pco2(Trange),Trange,p)
    return T

Both Formulas above are to calculate the equilibrium vapor pressure and temperature for carbon dioxide

In [5]:
# Function: ls2sol
# Purpose: convert Mars "Lsubs" to sol (day of year)
# Input: 
#    ls = areocentric longitude of the Sun [array]
#    n = degree of polynomial fit [scalar]
def ls2sol(ls, n):
    # data
    lsdata = (0,30,60,90,120,150,180,210,240,270,300,330,360) # L_s array
    soldata = (0,61.2,126.6,193.3,257.8,317.5,371.9,421.6,468.5,514.6,562.0,612.9,668.6) # Sol array
    
    # polynomial fit
    p = np.polyfit(lsdata, soldata, n)
    
    # sol for Ls input
    sol = np.polyval(p, ls)
    
    # return result
    return sol

Formula to convert solar longitude to Sols

In [6]:
df['Sol'] = ls2sol(df['L_s'],8)
dfday = df[df['LTST'] >= .5]
dfnight = df[df['LTST'] <= .5]

In [7]:
cond = (df['MY'],df['L_s'],pco2(df['T_surf']), df['p_surf'], tco2(df['p_surf']), df['T_surf'], df['LTST'], df['Surf_lat'], df['Surf_lon'], df['Dust_column'] , df['Sol'] )
cond_right= np.transpose(cond)
CO2 = pd.DataFrame(list(cond_right))
CO2.columns = ['MY','L_s','Eq_Vap_P','p_surf','Frost_T','T_surf','LTST', 'Surf_lat', 'Surf_lon', 'Dust_column', 'Sol']

In [8]:
newCO2 = CO2[CO2['Eq_Vap_P'] <= CO2['p_surf']]

In [9]:
CO2day = CO2[CO2['LTST'] >= 0.5]
CO2night = CO2[CO2['LTST'] <= 0.5]

In [10]:
def MYretrieval(dataframe):
    dataframes = []
    for i in range(8):
        dataframes.append(dataframe[(dataframe['MY'] == i + 28)])
    return dataframes

In [11]:
def CreateSubFrame(MarsDF, RanL_s, RanLat, RanLong):
    '''Function to divide previously created data frames in smaller data frames by providing ranges
    Input:
    MarsYearDF: Mars Data frames 
    RanL_s: range of Solar Longitude example [0,360]
    RanAlbedo: Albedo Range example [0.5,1.0]
    RanSZA Solar Zenith angle range example [0,45]
    RanLat and RanLong are set as default
    Output: Several Subframes'''
    L_s_MYDF = MarsDF['L_s']
    Lat_MYDF = MarsDF['Surf_lat']
    Long_MYDF = MarsDF['Surf_lon']
    MY_SUB = MarsDF[(L_s_MYDF <= RanL_s[-1])  & (L_s_MYDF >= RanL_s[0]) &
                        (Lat_MYDF <= RanLat[-1] ) & (Lat_MYDF >= RanLat[0]) & 
                        (Long_MYDF <= RanLong[-1]) & (Long_MYDF >= RanLong[0])]
    return MY_SUB

In [12]:
def column(MYDict, Parameter):
    '''Function to run statistics on data on the subframes from the dictionaries created by the SubDict function
    Input:
    MYDict: Dictionary Created by the SubDict Function
    Output:
    SubAvg: Average of the Albedo for each subframe in the dictionary
    SubStd: Standard Deviation of the Albedo for each subframe in the dictionary
    Sub_L_s: Average of the  Solar Longitude range for each subframe in the dictionary'''
    SubframeNum = len(MYDict.keys())
    SubTemp = []
    SubPres =[]
    SubMY = []
    SubLat =[]
    SubLong = []
    SubDust =[]
    SubSol = []
    SubStr = 'DataFrame{}'
    for i in range(SubframeNum):
        Subframe = MYDict[SubStr.format(i)]
        SubTemp.append((Subframe['T_surf'].values))
        SubPres.append((Subframe['p_surf'].values))
        SubMY.append((Subframe['MY'].values))
        SubLat.append((Subframe['Surf_lat'].values))
        SubLong.append((Subframe['Surf_lon'].values))
        SubDust.append((Subframe['Dust_column'].values))
        SubSol.append((Subframe['Sol'].values))
    SubTemp = np.array(SubTemp)
    SubPres = np.array(SubPres)
    SubMY = np.array(SubMY)
    SubLat = np.array(SubLat)
    SubLong = np.array(SubLong)
    SubDust = np.array(SubDust)
    SubSol = np.array(SubSol)
    if Parameter =='Temperature':
        return SubTemp
    elif Parameter =='Pressure': 
        return SubPres
    elif Parameter =='Martian Year':
        return SubMY 
    elif Parameter =='Latitude':
        return SubLat 
    elif Parameter =='Longitude':
        return SubLong
    elif Parameter =='Dust Column':
        return SubDust
    elif Parameter =='Sol':
        return SubSol 

In [13]:
def IRFlux(T):
    '''Calculate the luminosity of the thing.
    Input:
    T = Temperature
    Output:
    boltz = luminosity'''
    sigma = 5.67e-8 # W/m**2/K**4
    boltz = sigma * T**4
    return boltz

In [14]:
def emi(temp,frosttemp):
    sigma = 5.67e-8
    a = IRFlux(temp)
    b = IRFlux(frosttemp)
    emi = a/b
    return emi

In [15]:
def square(lat,lon,size=59.157935, type = 'km'): # 1 degree latitude (Co-created with Tyler Horvath)
    '''Function to draw equal length trapezoid that are latitude dependent
    Inputs:
    Latitude = array or float, latitude of the locations
    Longitude = array or float, longitude of the locations
    Size = size of the box default is equal to one degree by one degree could be kilometers or degrees
    type = string, if input is km the size argument is inputed as kilometers else, deg is inputed as degrees
    Outputs:
    lower_lat = lower latitude 
    upper_lat = upper latitude 
    right_lon = right longitude corner
    left_lon = left longitude corner'''
    radius = 3389.5 # mars radius in km
    d2r = np.pi/180
    if type == 'km': # treat size input as km if >= 59....
        lat_deg = size*180/(np.pi*radius)
        upper_lat = lat + .5 * lat_deg
        lower_lat = lat - .5 * lat_deg
        lon_deg = size / (np.pi * np.cos(upper_lat*d2r) / 180)
        left_lon = lon - .5 * lon_deg * size
        right_lon = lon + .5 * lon_deg * size
        return(lower_lat, upper_lat, right_lon, left_lon)
    elif type == 'deg':
        upper_lat = lat + .5 * size
        lower_lat = lat - .5 * size
        leng = radius * np.pi * np.cos(upper_lat*d2r)*2
        lon_deg = 59.157935*size/leng*360
        left_lon = lon - .5 * lon_deg 
        right_lon = lon + .5 * lon_deg
        return(lower_lat, upper_lat, right_lon, left_lon)

In [16]:
def xerr(a,b,covmat):
    '''Function to calculate the x-intercept of a fitted line in the form y = ax+b with its error
    Inputs:
    a = float, the slope of the fitted line
    b = float, the x intercept of the line
    covmat = array, the covariance matrix of the fitted line
    Outputs:
    x = float, the x-intercept of the line
    stdx = float , error of the line '''
    x = -b/a
    stdx = x*np.sqrt((covmat[1,1]/(b**2))+(covmat[0,0]/(a**2))-2*((covmat[0,1]/(a*b))))
    return x, stdx

In [361]:
def LocFunc(lat,lon,size=59.157935,type = 'km'):
    Location = square(lat, lon, size=2, type = 'deg')
    Day = MYretrieval(dfday)
    Night = MYretrieval(dfnight)
    CO2D = MYretrieval(CO2day)
    CO2N = MYretrieval(CO2night)
    for i in range(len(Day)):
        Day[i].insert(10 , 'Frost_T', CO2D[i]['Frost_T'])
        Night[i].insert(10 , 'Frost_T', CO2D[i]['Frost_T'])
    LocationDay = []
    LocationNight =[]
    for i in range(len(Day)):
        LocationDay.append(CreateSubFrame(Day[i], RanL_s=np.linspace(0,360), RanLat=np.linspace(Location[0],Location[1]), RanLong=np.linspace(Location[3],Location[2])))
        LocationNight.append(CreateSubFrame(Night[i], RanL_s=np.linspace(0,360), RanLat=np.linspace(Location[0],Location[1]), RanLong=np.linspace(Location[3],Location[2])))
    for i in range(len())
    MaxTempDay = []
    for i in range(len(LocationDay)):
        MaxTempDay.append(np.max(LocationDay[i]['T_surf'])[i])
    #FirstFrostDay = []
    #for i in range(len(LocationDay)):
    #    if lat >= 58.0: 
    #        FirstFrostDay.append(np.min(LocationDay[i]['L_s'][(LocationDay[i]['L_s'] >= 180.0) & (LocationDay[i]['T_surf'] <= LocationDay[i]['Frost_T'])]))
    #    elif (lat < 58.0 and lat > -58.0):
    #        FirstFrostDay.append(np.nan)
    #    elif lat <= -58.0: 
    #        FirstFrostDay.append(np.min(LocationDay[i]['L_s'][(LocationDay[i]['L_s'] <= 180.0) & (LocationDay[i]['T_surf'] <= LocationDay[i]['Frost_T'])]))
    #LastFrostDay = []
    #for i in range(len(LocationDay)):
    #    if lat >= 58.0: 
    #        FirstFrostDay.append(np.max(LocationDay[i]['L_s'][(LocationDay[i]['L_s'] >= 180.0) & (LocationDay[i]['T_surf'] <= LocationDay[i]['Frost_T'])]))
    #    elif (lat < 58.0 and lat > -58.0):
    #        FirstFrostDay.append(np.nan)
    #    elif lat <= -58.0: 
    #        FirstFrostDay.append(np.max(LocationDay[i]['L_s'][(LocationDay[i]['L_s'] <= 180.0) & (LocationDay[i]['T_surf'] <= LocationDay[i]['Frost_T'])]))
    ##FirstFrostNight = []
    ##FirstFrostNight = []
    #    #if lat >= 0
    #    #    FirstFrostNight.append(np.min(LocationNight[i]['L_s'][(LocationNight[i]['L_s'] >= 180.0) & (LocationNight[i]['T_surf'] <= LocationNight[i]['Frost_T'])]))
    #    #elif lat <= -58.0: 
    #    #    FirstFrostNight.append(np.min(LocationNight[i]['L_s'][(LocationNight[i]['L_s'] <= 180.0) & (LocationNight[i]['T_surf'] <= LocationNight[i]['Frost_T'])]))
    return LocationDay, LocationNight, MaxTempDay

SyntaxError: invalid syntax (<ipython-input-361-f04cbf79e9e9>, line 15)

In [359]:
BuzzelDay, BuzzelNight, FirstFrostDay = LocFunc(-58.0, -53.2, size=2, type = 'deg')

In [360]:
FirstFrostDay

[243.53200000000001,
 273.31599999999997,
 273.435,
 275.09100000000001,
 275.35700000000003,
 277.88099999999997,
 160.893,
 nan]

In [344]:
b = BuzzelDay[1]['T_surf']

In [345]:
type(a)

pandas.core.series.Series

In [309]:
FirstFrostNight

[nan, nan, nan, nan, nan, nan, nan, nan]

In [158]:
a = np.min(BuzzelDay[1]['L_s'][(BuzzelDay[1]['L_s'] <= 180.0) & (BuzzelDay[1]['T_surf'] <= BuzzelCO2D[1]['Frost_T'])])
b = np.min(BuzzelDay[1]['L_s'][(BuzzelDay[1]['L_s'] >= 180.0) & (BuzzelDay[1]['T_surf'] <= BuzzelCO2D[1]['Frost_T'])])

In [159]:
a

1.2654700000000001

In [160]:
b

187.99021999999999